In [4]:
from os import walk
import json
import re
import statistics
from utils import pmi, data_analysis, docs_distribution
from tqdm import tqdm

In [5]:
from Divergences import Jensen_Shannon
import numpy as np
from Scoring import compute_scores

In [6]:
my_path = './Recipe_dataset/RecipeFullDataset/'
save_path = './Recipe_dataset/Recipe_with_scores/'
filenames = next(walk(my_path), (None, None, []))[2]

### First estimation for the Individual term divergence for newness

In [66]:
thresholds_all = []
JS = Jensen_Shannon() 
means = []
std_devs = []
for i in tqdm(range(len(filenames))):
    file_recette = filenames[i]
    #Opening Json file with al lrecipes infos
    file_path = my_path + file_recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    KB_recettes, _ = data_analysis(recipe_dict)

    #Let's samples 15 recipe max per round -- we don't need more than that for threshold estimate
    lengths = min(len(KB_recettes), 30)
    for j in range(lengths):
        eval_recipe = KB_recettes[j]
        current_KB = [text for index, text in enumerate(KB_recettes) if index != i]

        KB_updated = current_KB + [eval_recipe]
        KB_matrix, KB_dist, NewKB_matrix, NewKB_dist, variation_dist = docs_distribution(KB_updated)
        
        JSD_vector = JS.linear_JSD(KB_dist, variation_dist) ## Here I get the individual divergence of each terms compare to the other -- now I want an average and a standard dev for that doc
        appear = []
        disappear = []
        for k in range(len(JSD_vector)):
            if JSD_vector[k] != 0:
                #print(variation_dist[k] / (KB_dist[k]+1e-10))
                if variation_dist[k] / (KB_dist[k]+1e-10) > 0:
                    if KB_dist[k] > 0:
                        appear.append(variation_dist[k] / (KB_dist[k]))
                if KB_dist[k] / (variation_dist[k]+1e-10) > 0:
                    if variation_dist[k] > 0:
                        disappear.append(KB_dist[k] / (variation_dist[k]))
        if len(appear) >1:
            avg_a = statistics.mean(appear)
            std_a = statistics.stdev(appear)
        if len(disappear) >1:
            avg_d = statistics.mean(disappear)
            std_d = statistics.stdev(disappear)
        means.append(avg_a)
        #means.append(avg_d)
        std_devs.append(std_a)
        #std_devs.append(std_d)
         
print('The average divergence of each terms for all documents in training is : ', statistics.mean(means))
print('The average standard deviation of the divergence of each terms for all documents in training is : ', statistics.mean(std_devs))

100%|████████████████████████████████████████████████████████████████████████████████| 507/507 [01:15<00:00,  6.72it/s]

The average divergence of each terms for all documents in training is :  14.640325295634126
The average standard deviation of the divergence of each terms for all documents in training is :  21.25346281695315


In [ ]:
thresholds_all = []
JS = Jensen_Shannon() 
means = []
std_devs = []
newness1s, newness2s, uniqueness1s, difference1s, new_surprise1s, dist_surprise1s, uniqueness2s, difference2s = [],[],[],[],[],[],[],[]
for i in tqdm(range(len(filenames))):
    file_recette = filenames[i]
    #Opening Json file with al lrecipes infos
    file_path = my_path + file_recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    KB_recettes, _ = data_analysis(recipe_dict)

    #Let's samples 15 recipe max per round -- we don't need more than that for threshold estimate
    lengths = min(len(KB_recettes), 10)
    if len(KB_recettes) >3:
        for j in range(lengths):
            eval_recipe = KB_recettes[j]
            current_KB = [text for index, text in enumerate(KB_recettes) if index != i]
            if len(current_KB) > 50:
                current_KB = current_KB[:50]
            newness1, _, uniqueness1, _, difference1, _, new_surprise1, _, dist_surprise1, _ = compute_scores(current_KB, eval_recipe, newness_type='div', uniq_type='shift', diff_type='local')
            newness2, _, uniqueness2, _, difference2, _, _, _, _, _ = compute_scores(current_KB, eval_recipe, newness_type='prob', uniq_type='dist', diff_type='global')
            
            newness1s.append(newness1)
            newness2s.append(newness2)
            uniqueness1s.append(uniqueness1)
            uniqueness2s.append(uniqueness2)
            difference1s.append(difference1)
            difference2s.append(difference2)
            new_surprise1s.append(new_surprise1)
            dist_surprise1s.append(dist_surprise1)

In [22]:
print('The average newness 1 : ', statistics.mean(newness1s))
print('The average standard deviation newness 1 : ', statistics.stdev(newness1s))
print('=====================================================')
print('The average newness 2 : ', statistics.mean(newness2s))
print('The average standard deviation newness 2 : ', statistics.stdev(newness2s))
print('=====================================================')
print('The average uniqueness 1 : ', statistics.mean(uniqueness1s))
print('The average standard deviation uniqueness 1 : ', statistics.stdev(uniqueness1s))
print('=====================================================')
print('The average uniqueness 2 : ', statistics.mean(uniqueness2s))
print('The average standard deviation uniqueness 2 : ', statistics.stdev(uniqueness2s))
print('=====================================================')
print('The average difference 1 : ', statistics.mean(difference1s))
print('The average standard deviation difference 1 : ', statistics.stdev(difference1s))
print('=====================================================')
print('The average difference 2 : ', statistics.mean(difference2s))
print('The average standard deviation difference 2 : ', statistics.stdev(difference2s))
print('=====================================================')
print('The average new_surprise : ', statistics.mean(new_surprise1s))
print('The average standard deviation new_surprise : ', statistics.stdev(new_surprise1s))
print('=====================================================')
print('The average dist_surprise : ', statistics.mean(dist_surprise1s))
print('The average standard deviation dist_surprise : ', statistics.stdev(dist_surprise1s))

The average newness 1 :  9.787785635152834e-05
The average standard deviation newness 1 :  0.0006823768444105392
The average newness 2 :  0.0006014464397221442
The average standard deviation newness 2 :  0.000429302944878075
The average uniqueness 1 :  0.07555522199120962
The average standard deviation uniqueness 1 :  0.027067390553279926
The average uniqueness 2 :  0.3413960451113142
The average standard deviation uniqueness 2 :  0.09305118657712405
The average difference 1 :  0.010468712824173209
The average standard deviation difference 1 :  0.07305834341965008
The average difference 2 :  0.1099779862216786
The average standard deviation difference 2 :  0.15392070117525491
The average new_surprise :  0.0016093334472012499
The average standard deviation new_surprise :  0.005274331203062446
The average dist_surprise :  0.0004865002584515531
The average standard deviation dist_surprise :  0.001043113510080868
